# iTunes Media Store Data Analysis

## 1. Project Description

## 2. Setup

**Installing ipython-sql**

In [37]:
!pip3 install ipython-sql

**Loading the database**

In [5]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

**Getting familiar with the database**

In [6]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## 3. Best-Selling Genres

<br>Invoice Line table doesnt directly contain information on genres, however, 
<br>it has a foreign key to the track table from which the genre information can be retrieved.

In [63]:
%%sql
WITH 
    genre_count AS
    (
        SELECT g.name AS genre, COUNT(il.quantity) AS number_of_tracks_sold
        FROM invoice_line AS il
        INNER JOIN track AS t ON il.track_id = t.track_id
        INNER JOIN genre g ON t.genre_id = g.genre_id
        GROUP BY genre
        ORDER BY number_of_tracks_sold DESC
    ),
    total AS
    (
        SELECT COUNT(*) AS total_sold FROM invoice_line
    )

SELECT gc.genre AS genre, 
        gc.number_of_tracks_sold AS number_sold, 
        (CAST(gc.number_of_tracks_sold AS REAL) / t.total_sold)*100 AS percentage 
FROM genre_count gc, total t;

 * sqlite:///chinook.db
Done.


genre,number_sold,percentage
Rock,2635,55.3920538154299
Metal,619,13.012402774858103
Alternative & Punk,492,10.342652932520496
Latin,167,3.510615934412445
R&B/Soul,159,3.3424427159974774
Blues,124,2.606684885431995
Jazz,121,2.543619928526382
Alternative,117,2.4595333193188984
Easy Listening,74,1.5556022703384484
Pop,63,1.3243640950178683


> The 5 top-selling genres are Rock, Metal, Alternative & Punk, Latin and R&B/Soul. The store shall consider this info when buying new tracks.

## 4. Top Sales Agents

In [83]:
%%sql
WITH 
    customer_spent AS
    (
        SELECT c.customer_id AS customer, 
                SUM(i.total) AS total_spent
        FROM invoice i
        INNER JOIN customer c ON i.customer_id = c.customer_id
        GROUP BY c.customer_id
        ORDER BY total_spent DESC
    ),
    employee_customer AS
    (
        SELECT e.first_name || " " || e.last_name AS sales_agent_name,
                c.customer_id AS customer
        FROM employee e
        LEFT JOIN customer c ON c.support_rep_id = e.employee_id
    )

SELECT ec.sales_agent_name, SUM(cs.total_spent) AS total_sales
FROM customer_spent cs
INNER JOIN employee_customer ec ON ec.customer = cs.customer
GROUP BY ec.sales_agent_name
ORDER BY total_sales DESC;

 * sqlite:///chinook.db
Done.


sales_agent_name,total_sales
Jane Peacock,1731.5099999999998
Margaret Park,1584.0
Steve Johnson,1393.92


> The top-performing Sales Agent is Jane. 

## 5. Analysis of Sales by Country

## 6. Album vs Individual Track Sales